In [ ]:
# sqlalchemy ORM 다루기
# asyncpg는 Python의 비동기 프로그래밍을 위한 PostgreSQL 데이터베이스 라이브러리

In [3]:
# 엔진설정 : DB와 연결 설정
from sqlalchemy import create_engine
engine = create_engine('sqlite:///example.db', echo=True)  # echo=True는 실행된 SQL을 출력합니다.

In [25]:
# 모델 정의 : DB 테이블과 매핑될 클래스를 정의
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'  # 사용할 테이블 이름
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)

# class Address(Base):
#     __tablename__ = 'addresses'
#     id = Column(Integer, primary_key=True)
#     email_address = Column(String, nullable=False)
#     user_id = Column(Integer, ForeignKey('users.id'))
#     user = relationship("User", back_populates="addresses")

C:\Users\195452\AppData\Local\Temp\ipykernel_30076\1574990273.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [28]:
# 테이블 생성
Base.metadata.create_all(engine)  # 데이터베이스에 테이블을 생성합니다.

2024-05-16 14:42:42,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:42:42,093 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-05-16 14:42:42,095 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-16 14:42:42,105 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2024-05-16 14:42:42,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-16 14:42:42,116 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
# 세션관리 - DB와의 모든 상호 작용은 세션을 통해 처리
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [30]:
# 데이터 추가 by 세션
new_user = User(name="John", fullname="John Doe")
session.add(new_user)
session.commit()  # 변경사항을 데이터베이스에 커밋합니다.

2024-05-16 14:42:50,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:42:50,641 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2024-05-16 14:42:50,644 INFO sqlalchemy.engine.Engine [generated in 0.00292s] ('John', 'John Doe')
2024-05-16 14:42:50,664 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
new_user = User(name="Hyunsu", fullname="Hyunsu Shin")
session.add(new_user)
session.commit()  # 변경사항을 데이터베이스에 커밋합니다.

2024-05-16 14:32:41,561 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2024-05-16 14:32:41,564 INFO sqlalchemy.engine.Engine [cached since 147.5s ago] ('Hyunsu', 'Hyunsu Shin')
2024-05-16 14:32:41,579 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
# 데이터 조회 by 세션
# 이름으로 필터링하여 사용자 조회
filtered_users = session.query(User).filter(User.name == "John").all()
for user in filtered_users:
    print(user.name, user.fullname)

2024-05-16 14:31:32,748 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:31:32,781 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2024-05-16 14:31:32,789 INFO sqlalchemy.engine.Engine [generated in 0.00783s] ('John',)
John John Doe


In [11]:
# 결과를 정렬하여 조회
sorted_users = session.query(User).order_by(User.fullname).all()
for user in sorted_users:
    print(user.name, user.fullname)

2024-05-16 14:32:46,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:32:46,045 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users ORDER BY users.fullname
2024-05-16 14:32:46,048 INFO sqlalchemy.engine.Engine [cached since 38.98s ago] ()
Hyunsu Hyunsu Shin
John John Doe


In [12]:
# 데이터 업데이트
user.name = "Jonathan"
session.commit()  # 업데이트한 내용을 커밋합니다.

2024-05-16 14:34:06,858 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2024-05-16 14:34:06,861 INFO sqlalchemy.engine.Engine [generated in 0.00300s] ('Jonathan', 1)
2024-05-16 14:34:06,871 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
searched_users = session.query(User).all()
for user in searched_users:
    print(user.name, user.fullname)

2024-05-16 14:34:56,264 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:34:56,275 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2024-05-16 14:34:56,278 INFO sqlalchemy.engine.Engine [generated in 0.00392s] ()
Jonathan John Doe
Hyunsu Hyunsu Shin


In [16]:
# 데이터 삭제
session.delete(user)
session.commit()  # 삭제를 커밋합니다.

2024-05-16 14:36:28,181 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2024-05-16 14:36:28,183 INFO sqlalchemy.engine.Engine [cached since 46.02s ago] (1,)
2024-05-16 14:36:28,196 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
searched_users = session.query(User).all()
for user in searched_users:
    print(user.name, user.fullname)

2024-05-16 14:36:30,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:36:30,761 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2024-05-16 14:36:30,769 INFO sqlalchemy.engine.Engine [cached since 94.49s ago] ()


In [26]:
# 관계 매핑 : 테이블 간의 관계 설정
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

In [27]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

In [31]:
# 사용자에게 주소 추가
new_address = Address(email_address="john@example.com", user_id=new_user.id)
session.add(new_address)
session.commit()

2024-05-16 14:43:00,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:43:00,252 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.id = ?
2024-05-16 14:43:00,256 INFO sqlalchemy.engine.Engine [generated in 0.00443s] (2,)
2024-05-16 14:43:00,271 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2024-05-16 14:43:00,276 INFO sqlalchemy.engine.Engine [generated in 0.00467s] ('john@example.com', 2)
2024-05-16 14:43:00,288 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
# 사용자의 모든 주소 조회
user_addresses = session.query(Address).filter_by(user_id=new_user.id).all()
for address in user_addresses:
    print(address.email_address)

2024-05-16 14:43:04,691 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:43:04,695 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.id = ?
2024-05-16 14:43:04,704 INFO sqlalchemy.engine.Engine [cached since 4.452s ago] (2,)
2024-05-16 14:43:04,715 INFO sqlalchemy.engine.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.user_id = ?
2024-05-16 14:43:04,719 INFO sqlalchemy.engine.Engine [generated in 0.00305s] (2,)
john@example.com


In [33]:
# 전체 사용자 조회
# SQLAlchemy 세션 설정
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# 모든 사용자 조회
users = session.query(User).all()
for user in users:
    print(user.name, user.fullname)

2024-05-16 14:43:22,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 14:43:22,183 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2024-05-16 14:43:22,186 INFO sqlalchemy.engine.Engine [generated in 0.00303s] ()
John John Doe
John John Doe


연습문제 
문제 1: 개발자 포트폴리오 프로젝트 관리
  개발자 포트폴리오 웹사이트를 위한 DB에 프로젝트 정보를 추가하는 기능 구현. 각 프로젝트는 이름(`name`), 설명(`description`), 기술 스택(`technologies`), 시작 날짜(`start_date`), 완료 날짜(`end_date`)를 포함

- 구현할 기능:
  a. `Project` 모델 생성 및 초기화.
  b. 사용자 입력을 받아 새 프로젝트 객체를 데이터베이스에 추가.
  c. 프로젝트를 세션에 추가하고 커밋.

In [61]:
session.close()
session = Session()

In [63]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date

Base = declarative_base()

class Project(Base):
    __tablename__ = 'projects'  # 사용할 테이블 이름
    id = Column(Integer, primary_key=True)
    name = Column(String)
    description = Column(String)
    technologies = Column(String)
    #start_date = Column(Date)
    #end_date = Column(Date)

C:\Users\195452\AppData\Local\Temp\ipykernel_30076\3202908129.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [64]:
Base.metadata.create_all(engine)  # 데이터베이스에 테이블을 생성합니다.

2024-05-16 15:51:46,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 15:51:46,337 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2024-05-16 15:51:46,340 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-16 15:51:46,344 INFO sqlalchemy.engine.Engine COMMIT


In [65]:
def add_pj(name, description, technologies):
  new_pj = Project(name=name, description=description, technologies=technologies)
  session.add(new_pj)
  session.commit()

In [66]:
add_pj('쇼핑몰 만들기', '옷', 'Django')

2024-05-16 15:51:54,671 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 15:51:54,676 INFO sqlalchemy.engine.Engine INSERT INTO projects (name, description, technologies) VALUES (?, ?, ?)
2024-05-16 15:51:54,680 INFO sqlalchemy.engine.Engine [generated in 0.00455s] ('쇼핑몰 만들기', '옷', 'Django')
2024-05-16 15:51:54,695 INFO sqlalchemy.engine.Engine COMMIT


문제 2: 개발자 포트폴리오 프로젝트 조회
 개발자 포트폴리오 웹사이트에서 모든 프로젝트의 상세 정보를 조회하는 기능을 구현. 각 프로젝트의 모든 필드 정보를 출력.

- 구현할 기능:
a. 세션을 사용하여 모든 프로젝트 조회.
b. 각 프로젝트의 상세 정보를 출력.

In [67]:
def list_pj():
  projects = session.query(Project).all()
  for pj in projects:
    print(f"{pj.name}, {pj.description}, {pj.technologies}")

In [72]:
list_pj()

2024-05-16 15:54:10,047 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 15:54:10,048 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.name AS projects_name, projects.description AS projects_description, projects.technologies AS projects_technologies 
FROM projects
2024-05-16 15:54:10,051 INFO sqlalchemy.engine.Engine [cached since 109.6s ago] ()
쇼핑몰 만들기, 옷 + 신발, 새로운 기술


### 문제 3: 개발자 포트폴리오 프로젝트 업데이트
 기존 프로젝트 정보를 업데이트하는 기능을 구현. 프로젝트 ID를 입력받아 해당 프로젝트의 기술 스택과 설명을 업데이트

**구현할 기능**:
1. 프로젝트 ID로 프로젝트 조회.
2. 기술 스택과 설명을 사용자 입력으로 업데이트.
3. 데이터베이스에 변경 사항 커밋.

In [69]:
def update_pj(project_id, technologies, description):
  pj = session.query(Project).filter(Project.id == project_id).first()
  if pj:
      pj.technologies = technologies
      pj.description = description
      session.commit()
  else:
    print("프로젝트가 없습니다.")

In [71]:
update_pj(1, "새로운 기술", "옷 + 신발")

2024-05-16 15:54:03,271 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.name AS projects_name, projects.description AS projects_description, projects.technologies AS projects_technologies 
FROM projects 
WHERE projects.id = ?
 LIMIT ? OFFSET ?
2024-05-16 15:54:03,273 INFO sqlalchemy.engine.Engine [cached since 6.164s ago] (1, 1, 0)
2024-05-16 15:54:03,279 INFO sqlalchemy.engine.Engine UPDATE projects SET description=?, technologies=? WHERE projects.id = ?
2024-05-16 15:54:03,280 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ('옷 + 신발', '새로운 기술', 1)
2024-05-16 15:54:03,289 INFO sqlalchemy.engine.Engine COMMIT
